# Problem set 3

- **Student:** Lucas Emanuel Resck Domingues
- **Professor:** Luciano Guimarães
- **FGV-EMAp**

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-9---Integer-Programming" data-toc-modified-id="Chapter-9---Integer-Programming-1">Chapter 9 - Integer Programming</a></span><ul class="toc-item"><li><span><a href="#Problem-1" data-toc-modified-id="Problem-1-1.1">Problem 1</a></span></li><li><span><a href="#Problem-3" data-toc-modified-id="Problem-3-1.2">Problem 3</a></span></li></ul></li><li><span><a href="#Reference" data-toc-modified-id="Reference-2">Reference</a></span></li></ul></div>

## Chapter 9 - Integer Programming

### Problem 1

As the leader of an oil-exploration drilling venture, you must determine the least-cost selection of 5 out of 10 possible sites.
Label the sites $S_1, S_2, \cdots, S_{10}$, and the exploration costs associated with each as $C_1, C_2, \cdots, C_{10}$.
Regional development restrictions are such that:

1. Evaluating sites $S_1$ and $S_7$ will prevent you from exploring site $S_8$.
2. Evaluating site $S_3$ or $S_4$ prevents you from assessing site $S_5$.
3. Of the group $S_5, S_6, S_7, S_8$, only two sites may be assessed.

Formulate an integer program to determine the minimum-cost exploration scheme that satisfies these restrictions.

**Solution:** Let $S_1, \cdots, S_{10} \in \{0, 1\}$ indicate if the respective site is selected.
We ask that
$$\sum_{i = 1}^{10} S_i = 5$$
so that just 5 of the 10 possible sites are selected.
Our objective is to minimize the cost:
$$\sum_{i = 1}^{10} C_ i S_i.$$

The regional development restrictions can be built this way:
- If $S_1$ and $S_7$ are selected, $S_8$ can't: $\ \ S_8 \le 2 - (S_1 + S_7)$
- If $S_3$ or $S_4$ is selected, $S_5$ can't: $\ \ S_5 \le 1 - S_3$, $\ \ S_5 \le 1 - S_4$
- Only two of $S_5, S_6, S_7, S_8$ can be selected: $S_5 + S_6 + S_7 + S_8 \le 2$

Let's see an example.

In [1]:
using Cbc, JuMP

m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0));

Let's consider $C_i = 1$, for simplicity. We create the variables:

In [2]:
@variable(m, S[1:10], Bin)

10-element Array{VariableRef,1}:
 S[1]
 S[2]
 S[3]
 S[4]
 S[5]
 S[6]
 S[7]
 S[8]
 S[9]
 S[10]

We add the objective:

In [3]:
@objective(m, Min, sum(S))

S[1] + S[2] + S[3] + S[4] + S[5] + S[6] + S[7] + S[8] + S[9] + S[10]

The restrictions:

In [4]:
# Just 5 selected
@constraint(m, sum(S) == 5)

# Regional restrictions
@constraint(m, S[8] ≤ 2 - (S[1] + S[7]))
@constraint(m, S[5] ≤ 1 - S[3])
@constraint(m, S[5] ≤ 1 - S[4])
@constraint(m, S[5] + S[6] + S[7] + S[8] ≤ 2);

We optimize it!

In [5]:
optimize!(m)
print(value.(S))

[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0]

It makes sense :)

### Problem 3

The marketing group of A. J. Pitt Company is considering the options available for its next advertising campaign program.
After a great deal of work, the group has identified a selected number of options with the characteristics shown in the accompanying table.

| | TV | Trade magazine | Newspaper | Radio | Popular magazine | Promotional campaign | Total resource available |
|--- | --- | --- | --- | --- | --- | --- | --- |
| **Customers reached** | 1,000,000 | 200,000 | 300,000 | 400,000 | 450,000 | 450,000 | - |
| **Cost ($)** | 500,000 | 150,000 | 300,000 | 250,000 | 250,000 | 100,000 | 1,800,000 |
| **Designers needed (man-hours)** | 700 | 250 | 200 | 200 | 300 | 400 | 1,500 |
| **Salesmen needed (man-hours)** | 200 | 100 | 100 | 100 | 100 | 1,000 | 1,200 |

The objective of the advertising program is to maximize the number of customers reached, subject to the limitation of resources (money, designers, and salesman) given in the table above.
In addition, the following constraints have to be met:

1. If the promotional campaign is undertaken, it needs either a radio or a popular magazine campaign effort to
support it.
2. The firm cannot advertise in both the trade and popular magazines.

Formulate an integer-programming model that will assist the company to select an appropriate advertising campaign
strategy.

**Solution:** Let $x_1, \cdots, x_6$ be binary variables indicating advertising in $\text{TV}, \cdots, \text{Promotional campaign}$. Consider the above table as a matrix $C$. We want to maximize the number of customers reached:

$$\sum_{i = 1}^6 x_i C_{1i}$$

We have constraints of cost, designers and salesmen:

$$\sum_{i = 1}^6 x_i C_{ji} \le C_{j7}, \ \ j = 2, 3, 4$$

Two more constraints need to be met:

- Promotional campaign implies radio or popular magazine: $x_6 \le x_4 + x_5$
- The firm cannot advertise in both the trade and popular magazines: $x_2 + x_5 \le 1$

Let's implement it!

Our matrix $C$:

In [6]:
C = [
    [1000000, 200000, 300000, 400000, 450000, 450000, 0],
    [500000, 150000, 300000, 250000, 250000, 100000, 1800000],
    [700, 250, 200, 200, 300, 400, 1500],
    [200, 100, 100, 100, 100, 1000, 1200]
];

In [7]:
m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0));

Our variables $x_i$:

In [8]:
@variable(m, x[1:6], Bin)

6-element Array{VariableRef,1}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]

The objective:

In [9]:
@objective(m, Max, sum(x.*C[1][1:6]))

1.0e6 x[1] + 200000 x[2] + 300000 x[3] + 400000 x[4] + 450000 x[5] + 450000 x[6]

The constraints:

In [10]:
# Cost, designers and salesmen
for j in [2,3,4]
    @constraint(m, sum(x.*C[j][1:6]) ≤ C[j][7])
end

# Aditional constraints
@constraint(m, x[6] ≤ x[4] + x[5])
@constraint(m, x[2] + x[5] ≤ 1)

x[2] + x[5] ≤ 1.0

Optimizing...

In [11]:
optimize!(m)
print(value.(x))

[1.0, 0.0, 1.0, 1.0, 1.0, 0.0]

## Reference

```bibtex
@book{bradley1977applied,
  title={Applied mathematical programming},
  author={Bradley, Stephen P and Hax, Arnoldo C and Magnanti, Thomas L},
  year={1977},
  publisher={Addison-Wesley}
}
```